In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ngmix
import numpy as np
import galsim

In [3]:
from ngmix.adaptive_prepsfmom import PrePSFAdmom
from ngmix.prepsfmom import PGaussMom

In [80]:
from ngmix import Jacobian, Observation

rng = np.random.RandomState()
image_size = 101
psf_image_size = 53
pixel_scale = 0.2
snr = 2e1
fwhm = 2.5
psf_fwhm = 0.9

cen = (image_size - 1)/2
psf_cen = (psf_image_size - 1)/2
gs_wcs = galsim.ShearWCS(
    pixel_scale, galsim.Shear(g1=-0.1, g2=0.06)).jacobian()
scale = np.sqrt(gs_wcs.pixelArea())
shift = rng.uniform(low=-scale/2, high=scale/2, size=2)
psf_shift = rng.uniform(low=-scale/2, high=scale/2, size=2)
xy = gs_wcs.toImage(galsim.PositionD(shift))
psf_xy = gs_wcs.toImage(galsim.PositionD(psf_shift))

jac = Jacobian(
    y=cen + xy.y, x=cen + xy.x,
    dudx=gs_wcs.dudx, dudy=gs_wcs.dudy,
    dvdx=gs_wcs.dvdx, dvdy=gs_wcs.dvdy)

psf_jac = Jacobian(
    y=psf_cen + psf_xy.y, x=psf_cen + psf_xy.x,
    dudx=gs_wcs.dudx, dudy=gs_wcs.dudy,
    dvdx=gs_wcs.dvdx, dvdy=gs_wcs.dvdy)

gal = galsim.Gaussian(
    fwhm=fwhm
).shear(
    e1=-0.1, e2=0.2
).withFlux(
    400
).shift(
    dx=shift[0], dy=shift[1]
)

gal = galsim.Exponential(
    half_light_radius=fwhm,
).shear(
    e1=-0.1, e2=0.2
).withFlux(
    400
).shift(
    dx=shift[0], dy=shift[1]
)


psf = galsim.Gaussian(
    fwhm=psf_fwhm
).shear(
    g1=0.3, g2=-0.15
)
im = galsim.Convolve([gal, psf]).drawImage(
    nx=image_size,
    ny=image_size,
    wcs=gs_wcs
).array
noise = np.sqrt(np.sum(im**2)) / snr
wgt = np.ones_like(im) / noise**2

psf_im = psf.shift(
    dx=psf_shift[0], dy=psf_shift[1]
).drawImage(
    nx=psf_image_size,
    ny=psf_image_size,
    wcs=gs_wcs
).array

im_true = gal.drawImage(
    nx=image_size,
    ny=image_size,
    wcs=gs_wcs,
    method='no_pixel').array
obs_true = Observation(
    image=im_true,
    jacobian=jac,
)

obs = Observation(
    image=im + rng.normal(size=im.shape, scale=noise),
    weight=wgt,
    jacobian=jac,
    psf=Observation(image=psf_im, jacobian=psf_jac),
)

In [81]:
import pprint

mom_fwhm = 2

res = PrePSFAdmom(min_fwhm=mom_fwhm).go(obs)
pres = PGaussMom(fwhm=mom_fwhm).go(obs)
print("pgauss s/n:   ", pres["s2n"])
print("am pgauss s/n:", res["s2n"])
for key in pres.keys():
    if key in res:
        print("%s:\n    %s\n    %s" % (key, pprint.pformat(pres[key]), pprint.pformat(res[key])))

pgauss s/n:    15.22407107309938
am pgauss s/n: 20.398578516475002
flags:
    0
    0
flagstr:
    ''
    ''
flux:
    69.33414352040658
    178.0375527984145
mom:
    array([        nan,         nan,  0.59557113,  5.44429804, 70.3000323 ,
       69.33414352])
    array([          nan,           nan, -117.98256203,   41.37483867,
        621.99817567,  178.0375528 ])
mom_cov:
    array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        , 22.74556694, -1.11350998,  6.30568913,
        -3.92145246],
       [ 0.        ,  0.        , -1.11350998, 21.07872776, -3.15699107,
         1.96334851],
       [ 0.        ,  0.        ,  6.30568913, -3.15699107, 26.13590134,
         8.83124273],
       [ 0.        ,  0.        , -3.92145246,  1.96334851,  8.83124273,
        20.7411439 ]])
    array([[ 1.00000000e+00,  0.00000000e+00, 